In [ ]:
import mne
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from metadata import *

print(subject_list)
print(tmin, tmax)

In [ ]:
# FOR EACH SUBJECT, CREATE AND SAVE EPOCH OBJECT

alpha = 10 # filtering threshold

for subject in subject_list:
    print('##### Processing subject', subject, '...')

    print('# Reading data...')
    raw = mne.io.read_raw_fif(ica_after_files[subject], preload = True)
    raw.set_montage(biosemi_layout)
    events = mne.find_events(raw)

    print('# Epoching...')
    epochs = mne.Epochs(
        raw, 
        events = events, 
        tmin = tmin, 
        tmax = tmax, 
        baseline = (tmin, 0), 
        event_id = event_id
    )

    print('# Dropping outliers...')
    epoch_data = epochs.get_data()[:, :len(EEG_channels), :] # [n_events, n_channels, n_time]
    trial_ranges = np.max(epoch_data, axis = 2)-np.min(epoch_data, axis = 2)
    ranges_zscore = stats.zscore(trial_ranges, axis = 0)
    bad_trials = np.any(ranges_zscore > alpha, axis = 1)
    print("# bad_trials numbers", bad_trials.sum())
    epochs.drop(np.where(bad_trials == 1)[0])

    print('# Saving subject', subject, '...')
    epochs.save(epoch_files[subject])

print('***** ALL DONE *****')